## ML Analysis

First, we import the `TensorFlow` and `numpy` libraries.

In [30]:
import tensorflow as tf
import pandas as pd
import numpy as np

We want our neural network to "learn" the relationship between list of inputs and list of outputs

**Example**
Here `x` represents the input array, which is a 2D array with 4 column (input) variables. `y` represents the output variable, which is 1D array with 1 output value per row in the `x` array.

In [31]:
x = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0, 2.0, 1.0, 4.5, 2.3, 6.7, 1.0], dtype=float)
y = np.array([-3.0, -1.0, 1.0, 3.0, 4.0, 2.0, 1.0, 3.0, 4.0, 2.0, 1.0, 2.0], dtype=float)

### Importing data

In [32]:
data = pd.read_csv("data.csv")
data = data.loc[:,['resp_id', 'age', 'gender', 'student', 'wfh_now', 'prod_change']]
data.columns = data.columns.to_series().apply(lambda x: x.strip())
data

,resp_id,age,gender,student,wfh_now,prod_change
0,11,44,Male,No,Yes,Increased somewhat
1,29,39,Male,No,Yes,Decreased somewhat
2,30,49,Female,No,Yes,Decreased somewhat
3,31,27,Male,No,Yes,Decreased somewhat
4,34,32,Female,"Yes, Full-time",Yes,In some ways it has increased and in other way...
...,...,...,...,...,...,...
2900,9319,67,Male,No,Question not displayed to respondent,Question not displayed to respondent
2901,9322,47,Female,No,No,Increased somewhat
2902,9323,59,Male,No,No,About the same
2903,9325,60,Female,No,No,Decreased somewhat


### Cleaning up data

We now have to convert the data from text to decimal values, to provide consistency over all the columns which will help the model fit our data.

Gender: 0 Male, 1 Female

Student: 0 Not a student, 1 Student Full time, 2 Student Part time

WFH_Now: 0 No, 1 Yes

Prod_Change: 0 Decreased Significantly, 1 Decreased Somewhat, 2 Both, 3 Same, 4 Increased somewhat, 5 Increased significantly

In [33]:
Genders = {
  'Male' : 0,
  'Female' : 1,
}

Student = {
  'No' : 0,
  'Yes' : 1,
}

WFH_Now = {
  'No' : 0,
  'Yes' : 1,
}

Prod_Change = {
  'Decreased significantly': 0,
  'Decreased somewhat': 1,
  'Both': 2,
  'Same': 3,
  'Increased somewhat': 4,
  'Increased significantly': 5,
}

for index, row in data.iterrows():
  # print(index)
  gender = row['gender']
  student = row['student']
  wfh_now = row['wfh_now']
  prod_change = row['prod_change']

  data.loc[index, 'gender'] = Genders.get(gender, -1)
  data.loc[index, 'student'] = Student.get(student, -1)
  data.loc[index, 'wfh_now'] = WFH_Now.get(wfh_now, -1)
  data.loc[index, 'prod_change'] = Prod_Change.get(prod_change, -1)
  
  # drop all the rows that have missing values (aka have a -1 in any column)
  if (data.loc[index, 'gender'] == -1 or data.loc[index, 'student']==-1 or data.loc[index, 'wfh_now']==-1 or data.loc[index, 'prod_change']==-1):
    data.drop(labels=index, axis=0, inplace=True)

data

,resp_id,age,gender,student,wfh_now,prod_change
0,11,44,0,0,1,4
1,29,39,0,0,1,1
2,30,49,1,0,1,1
3,31,27,0,0,1,1
10,46,63,1,0,1,4
...,...,...,...,...,...,...
2887,9280,61,1,0,1,4
2888,9281,65,1,0,1,4
2890,9286,67,0,0,1,4
2901,9322,47,1,0,0,4


We create the simplest possible neural network. It has 1 layer, that layer has 1 neuron, and the input is just 1 value.

In [34]:
model = tf.keras.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1])])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 1)                 2         
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


Next, we compile the neural network by specifying a **loss function** and an **optimization algorithm**.

In [35]:
model.compile(optimizer='sgd', loss='mean_squared_error')

Let's now **train** our neural network to fit the data. The neural network will try to guess the relationship between the values in `x` and the values in `y`. The loss function will measure how good or how bad this guess is and, based on this, the optimization algorithm will make another guess. We then repeat this process for a certin number of iterations (`epochs`).

In [36]:
x = data.loc[:,['gender']]
y = data.loc[:,['prod_change']]

x = np.asarray(x).astype('float32')
y = np.asarray(y).astype('float32')

model.fit(x, y, epochs=20)

Epoch 1/20
20/20 [==============================] - 0s 491us/step - loss: 5.2910
Epoch 2/20
20/20 [==============================] - 0s 488us/step - loss: 4.5662
Epoch 3/20
20/20 [==============================] - 0s 459us/step - loss: 4.2427
Epoch 4/20
20/20 [==============================] - 0s 450us/step - loss: 4.0728
Epoch 5/20
20/20 [==============================] - 0s 426us/step - loss: 3.9461
Epoch 6/20
20/20 [==============================] - 0s 417us/step - loss: 3.8441
Epoch 7/20
20/20 [==============================] - 0s 411us/step - loss: 3.7593
Epoch 8/20
20/20 [==============================] - 0s 404us/step - loss: 3.6860
Epoch 9/20
20/20 [==============================] - 0s 400us/step - loss: 3.6211
Epoch 10/20
20/20 [==============================] - 0s 396us/step - loss: 3.5670
Epoch 11/20
20/20 [==============================] - 0s 421us/step - loss: 3.5184
Epoch 12/20
20/20 [==============================] - 0s 413us/step - loss: 3.4733
Epoch 13/20
20/20 [======

Finally, let's test our neural network by using it to predict the value of `prod_change` for a previously unseen value of `gender` (for example, `gender=1`). **What do you think the value of `prod_change` will be?** 

In [37]:
print(model.predict([1.0,0.0]))

1/1 [==============================] - 0s 32ms/step
[[2.4382176]
 [1.9966186]]


This is a reasonable prediction since we only used binary gender as input and productivity levels (0-5) as output. And we currently assume no correlation between the two categories but we will improve our model but adding more features and improving the neural network.

### Reflection
* The most challenging part of this project is estimating individual productivity since there are many factors that could affect human behvaior, it's hard to draw conclusion, and it's also hard to use all the features from our survey data to simply predict productivity.
* Our initial insights is that workplace productivity is affected when people change scenes and switch between working in person to working from home. And the choice to work from home is fueled by other factors including education, income, job type, etc. 
* We don't have concrete results yet because we cleaned up the data and found out we might not have enough data of a certain feature to use it in the final model. We will have to tweak the model significantly.
* Biggest problems currently facing: Tweaking the neural network, training data and minimizing loss.
* We are on track to completing the project on time.
* Yes it's worth proceeding with the project because we have enough diverse set of features to draw important conclusions affecting individual's (and families) work from home productivity in general.

### Wave 1 Train

In [58]:
wave1 = pd.read_csv("Wave1_train.csv")      # All Data

wave1_train = wave1.iloc[:821,:]            # Training Data
wave1_validation = wave1.iloc[821:,:]       # Validation Data

wave1_test = pd.read_csv("Wave1_test.csv")  # Test Data
wave1_train.columns = wave1_train.columns.to_series().apply(lambda x: x.strip())
wave1_train.shape, wave1_validation.shape
wave1_train

,resp_id,WFH_PRE,Job_Clerical or administrative support,"Job_Manufacturing, construction, maintenance, or farming","Job_Professional, managerial, or technical",Job_Sales or service,Workload_increased,Workload_decreased,Increased_productivity,Decreased_productivity,hhveh_harm,age,gender,Number_bedrooms,Race_white,Gradutae_degree,High_income(LessThan_100K),More_income(LMoreThan_35K),ProEnvironment
0,11,0,0,0,0,0,1,0,1,0,2,44,1,5,1,0,0,0,1
1,29,1,0,0,1,0,0,0,0,1,1,39,1,2,1,1,0,1,1
2,30,0,0,0,1,0,0,0,0,1,4,49,0,4,1,0,0,1,1
3,31,1,0,0,1,0,0,0,0,1,1,27,1,1,1,1,0,1,1
4,34,1,0,0,1,0,0,1,0,0,0,32,0,5,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,3710,1,0,0,0,1,0,0,0,0,2,63,1,4,1,1,0,1,1
817,3714,0,1,0,0,0,0,0,0,0,1,62,1,3,1,0,0,0,1
818,3730,0,0,1,0,0,0,1,0,1,2,57,1,2,1,0,0,0,1
819,3733,1,0,0,1,0,0,0,0,0,2,34,0,3,1,1,0,1,0


We create the simplest possible neural network. It has 1 layer, that layer has 1 neuron, and the input is just 1 value.

#### Age Job Type Model

In [59]:
input1 = tf.keras.layers.Input(shape=(1,))
input2 = tf.keras.layers.Input(shape=(1,))
input3 = tf.keras.layers.Input(shape=(1,))
input4 = tf.keras.layers.Input(shape=(1,))
input5 = tf.keras.layers.Input(shape=(1,))


merged = tf.keras.layers.Concatenate(axis=1)([input1, input2, input3, input4, input5])
dense1 = tf.keras.layers.Dense(5, input_dim=1, activation=tf.keras.activations.sigmoid, use_bias=True)(merged)
output = tf.keras.layers.Dense(1, activation=tf.keras.activations.relu, use_bias=True)(dense1)
age_job_category_model = tf.keras.models.Model([input1, input2, input3, input4, input5], output)
age_job_category_model.compile(optimizer='sgd', loss='mean_squared_error', metrics=['accuracy'])
age_job_category_model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_36 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_37 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_38 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_39 (InputLayer)          [(None, 1)]          0           []                               
                                                                                            

In [60]:
# Train data
input1 = wave1_train.loc[:,['age']]
input2 = wave1_train.loc[:,['Job_Clerical or administrative support']]
input3 = wave1_train.loc[:,['Job_Manufacturing, construction, maintenance, or farming']]
input4 = wave1_train.loc[:,['Job_Professional, managerial, or technical']]
input5 = wave1_train.loc[:,['Job_Sales or service']]

input1 = np.asarray(input1).astype('float32')
input2 = np.asarray(input2).astype('float32')
input3 = np.asarray(input3).astype('float32')
input4 = np.asarray(input4).astype('float32')
input5 = np.asarray(input5).astype('float32')

y = wave1_train.loc[:,['WFH_PRE']]
y = np.asarray(y).astype('float32')

# Validation data
v_input1 = wave1_validation.loc[:,['age']]
v_input2 = wave1_validation.loc[:,['Job_Clerical or administrative support']]
v_input3 = wave1_validation.loc[:,['Job_Manufacturing, construction, maintenance, or farming']]
v_input4 = wave1_validation.loc[:,['Job_Professional, managerial, or technical']]
v_input5 = wave1_validation.loc[:,['Job_Sales or service']]

v_input1 = np.asarray(v_input1).astype('float32')
v_input2 = np.asarray(v_input2).astype('float32')
v_input3 = np.asarray(v_input3).astype('float32')
v_input4 = np.asarray(v_input4).astype('float32')
v_input5 = np.asarray(v_input5).astype('float32')

v_y = wave1_validation.loc[:,['WFH_PRE']]
v_y = np.asarray(v_y).astype('float32')

age_job_category_model.fit([input1, input2, input3, input4, input5],y, 
                        batch_size=36, epochs=100, validation_data=([v_input1, v_input2, v_input3, v_input4, v_input5], v_y))


Epoch 1/100
23/23 [==============================] - 0s 6ms/step - loss: 0.4884 - accuracy: 0.5116 - val_loss: 0.3971 - val_accuracy: 0.6029
Epoch 2/100
23/23 [==============================] - 0s 2ms/step - loss: 0.4884 - accuracy: 0.5116 - val_loss: 0.3971 - val_accuracy: 0.6029
Epoch 3/100
23/23 [==============================] - 0s 2ms/step - loss: 0.4884 - accuracy: 0.5116 - val_loss: 0.3971 - val_accuracy: 0.6029
Epoch 4/100
23/23 [==============================] - 0s 2ms/step - loss: 0.4884 - accuracy: 0.5116 - val_loss: 0.3971 - val_accuracy: 0.6029
Epoch 5/100
23/23 [==============================] - 0s 2ms/step - loss: 0.4884 - accuracy: 0.5116 - val_loss: 0.3971 - val_accuracy: 0.6029
Epoch 6/100
23/23 [==============================] - 0s 2ms/step - loss: 0.4884 - accuracy: 0.5116 - val_loss: 0.3971 - val_accuracy: 0.6029
Epoch 7/100
23/23 [==============================] - 0s 2ms/step - loss: 0.4884 - accuracy: 0.5116 - val_loss: 0.3971 - val_accuracy: 0.6029
Epoch 8/100
2

In [44]:
# Test data
t_input1 = wave1_test.loc[:,['age']]
t_input2 = wave1_test.loc[:,['Job_Clerical or administrative support']]
t_input3 = wave1_test.loc[:,['Job_Manufacturing, construction, maintenance, or farming']]
t_input4 = wave1_test.loc[:,['Job_Professional, managerial, or technical']]
t_input5 = wave1_test.loc[:,['Job_Sales or service']]

t_input1 = np.asarray(t_input1).astype('float32')
t_input2 = np.asarray(t_input2).astype('float32')
t_input3 = np.asarray(t_input3).astype('float32')
t_input4 = np.asarray(t_input4).astype('float32')
t_input5 = np.asarray(t_input5).astype('float32')

In [45]:
# prediction
age_job_category_model.predict([t_input1, t_input2, t_input3, t_input4, t_input5])

10/10 [==============================] - 0s 1ms/step


array([[0.49092886],
       [0.48997596],
       [0.49107453],
       [0.48268172],
       [0.48730424],
       [0.48997596],
       [0.4777086 ],
       [0.49102786],
       [0.49134395],
       [0.49127498],
       [0.49121407],
       [0.49127498],
       [0.49105105],
       [0.49133167],
       [0.49066594],
       [0.49107453],
       [0.48609582],
       [0.4900615 ],
       [0.49136767],
       [0.49134395],
       [0.49121407],
       [0.48879763],
       [0.49107453],
       [0.4858593 ],
       [0.49128994],
       [0.4913027 ],
       [0.48730424],
       [0.48663548],
       [0.4894394 ],
       [0.4911867 ],
       [0.48853442],
       [0.48997596],
       [0.4912577 ],
       [0.49131367],
       [0.4897448 ],
       [0.49107453],
       [0.49136326],
       [0.4910241 ],
       [0.48465696],
       [0.49061707],
       [0.4908177 ],
       [0.49115488],
       [0.48947564],
       [0.4911867 ],
       [0.49132302],
       [0.4858578 ],
       [0.49034497],
       [0.491

#### Degree or Pro Environment Model

In [61]:
input1 = tf.keras.layers.Input(shape=(1,))
input2 = tf.keras.layers.Input(shape=(1,))
input3 = tf.keras.layers.Input(shape=(1,))
input4 = tf.keras.layers.Input(shape=(1,))
input5 = tf.keras.layers.Input(shape=(1,))

merged = tf.keras.layers.Concatenate(axis=1)([input1, input2, input3, input4, input5])
dense1 = tf.keras.layers.Dense(5, input_dim=1, activation=tf.keras.activations.sigmoid, use_bias=True)(merged)
output = tf.keras.layers.Dense(1, activation=tf.keras.activations.relu, use_bias=True)(dense1)
degree_environment_model = tf.keras.models.Model([input1, input2, input3, input4, input5], output)
degree_environment_model.compile(optimizer='sgd', loss='mean_squared_error', metrics=['accuracy'])
degree_environment_model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_41 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_42 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_43 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_44 (InputLayer)          [(None, 1)]          0           []                               
                                                                                            

In [62]:
# Training Data
input1 = wave1_train.loc[:,['age']]
input2 = wave1_train.loc[:,['Workload_increased']]
input3 = wave1_train.loc[:,['Workload_decreased']]
input4 = wave1_train.loc[:,['Gradutae_degree']]
input5 = wave1_train.loc[:,['ProEnvironment']]

input1 = np.asarray(input1).astype('float32')
input2 = np.asarray(input2).astype('float32')
input3 = np.asarray(input3).astype('float32')
input4 = np.asarray(input4).astype('float32')
input5 = np.asarray(input5).astype('float32')

y = wave1_train.loc[:,['WFH_PRE']]
y = np.asarray(y).astype('float32')

# Validation Data
v_input1 = wave1_validation.loc[:,['age']]
v_input2 = wave1_validation.loc[:,['Workload_increased']]
v_input3 = wave1_validation.loc[:,['Workload_decreased']]
v_input4 = wave1_validation.loc[:,['Gradutae_degree']]
v_input5 = wave1_validation.loc[:,['ProEnvironment']]

v_input1 = np.asarray(v_input1).astype('float32')
v_input2 = np.asarray(v_input2).astype('float32')
v_input3 = np.asarray(v_input3).astype('float32')
v_input4 = np.asarray(v_input4).astype('float32')
v_input5 = np.asarray(v_input5).astype('float32')

v_y = wave1_validation.loc[:,['WFH_PRE']]
v_y = np.asarray(v_y).astype('float32')

degree_environment_model.fit([input1, input2, input3, input4, input5],y, batch_size=36, epochs=100, 
                        validation_data=([v_input1, v_input2, v_input3, v_input4, v_input5], v_y))

Epoch 1/100
23/23 [==============================] - 0s 5ms/step - loss: 0.2732 - accuracy: 0.5116 - val_loss: 0.2408 - val_accuracy: 0.6029
Epoch 2/100
23/23 [==============================] - 0s 2ms/step - loss: 0.2513 - accuracy: 0.5116 - val_loss: 0.2452 - val_accuracy: 0.6029
Epoch 3/100
23/23 [==============================] - 0s 3ms/step - loss: 0.2502 - accuracy: 0.5104 - val_loss: 0.2478 - val_accuracy: 0.6029
Epoch 4/100
23/23 [==============================] - 0s 2ms/step - loss: 0.2504 - accuracy: 0.5104 - val_loss: 0.2476 - val_accuracy: 0.6029
Epoch 5/100
23/23 [==============================] - 0s 2ms/step - loss: 0.2501 - accuracy: 0.5128 - val_loss: 0.2480 - val_accuracy: 0.6017
Epoch 6/100
23/23 [==============================] - 0s 2ms/step - loss: 0.2503 - accuracy: 0.5104 - val_loss: 0.2479 - val_accuracy: 0.6029
Epoch 7/100
23/23 [==============================] - 0s 2ms/step - loss: 0.2502 - accuracy: 0.5079 - val_loss: 0.2476 - val_accuracy: 0.6029
Epoch 8/100
2

In [ ]:
# Test data
t_input1 = wave1_test.loc[:,['age']]
t_input2 = wave1_test.loc[:,['Workload_increased']]
t_input3 = wave1_test.loc[:,['Workload_decreased']]
t_input4 = wave1_test.loc[:,['Gradutae_degree']]
t_input5 = wave1_test.loc[:,['ProEnvironment']]

t_input1 = np.asarray(t_input1).astype('float32')
t_input2 = np.asarray(t_input2).astype('float32')
t_input3 = np.asarray(t_input3).astype('float32')
t_input4 = np.asarray(t_input4).astype('float32')
t_input5 = np.asarray(t_input5).astype('float32')

In [49]:
# prediction
degree_environment_model.predict([t_input1, t_input2, t_input3, t_input4, t_input5])

10/10 [==============================] - 0s 628us/step


array([[0.48788524],
       [0.487656  ],
       [0.4879204 ],
       [0.48520166],
       [0.48683208],
       [0.487656  ],
       [0.48335648],
       [0.48789835],
       [0.48796362],
       [0.48795462],
       [0.4879452 ],
       [0.48795462],
       [0.48795402],
       [0.4879613 ],
       [0.48782486],
       [0.4879204 ],
       [0.48660904],
       [0.48767382],
       [0.48796582],
       [0.48796362],
       [0.4879452 ],
       [0.48731196],
       [0.4879204 ],
       [0.48579633],
       [0.48795676],
       [0.48795855],
       [0.48683208],
       [0.48660684],
       [0.48740232],
       [0.48794067],
       [0.48726445],
       [0.487656  ],
       [0.4879521 ],
       [0.48795992],
       [0.48759186],
       [0.4879204 ],
       [0.48796546],
       [0.4879105 ],
       [0.4857415 ],
       [0.48782074],
       [0.487867  ],
       [0.48793507],
       [0.48751497],
       [0.48794067],
       [0.48796117],
       [0.48633897],
       [0.4877534 ],
       [0.487

### Wave 2 Train

In [55]:
wave2 = pd.read_csv("Wave2_train.csv")      # All Data

train = wave2.iloc[:684,:]            # Training Data
validation = wave2.iloc[684:,:]       # Validation Data

test = pd.read_csv("Wave2_test.csv")  # Test Data
train.columns = train.columns.to_series().apply(lambda x: x.strip())
train.shape, validation.shape
train

,resp_id,wfh_expect,Job_Clerical or administrative support,"Job_Manufacturing, construction, maintenance, or farming","Job_Professional, managerial, or technical",Job_Sales or service,Workload_increased,Workload_decreased,Increased_productivity,Decreased_productivity,hhveh_harm,age,gender,Number_bedrooms,Race_white,Gradutae_degree,High_income(LessThan_100K),More_income(LMoreThan_35K)
0,1177,Yes,0,0,1,0,0,0,1,0,2,58,1,3,0,1,0,1
1,1188,Yes,0,0,1,0,0,0,1,0,1,66,0,2,1,0,0,0
2,1189,Yes,0,0,0,0,0,0,0,0,0,24,1,3,1,0,1,0
3,1194,Yes,0,0,1,0,0,0,0,0,1,31,1,3,1,1,0,1
4,1202,Yes,0,0,0,0,0,0,0,0,0,23,0,2,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,4593,No,0,0,0,1,0,1,0,1,4,59,0,3,1,0,0,1
680,4601,Yes,0,0,1,0,0,0,0,0,2,56,1,3,1,0,0,0
681,4613,Yes,0,0,0,1,1,0,1,0,2,66,0,4,1,1,0,1
682,4618,Yes,0,0,1,0,0,1,0,1,1,73,0,3,1,1,0,0


We create the simplest possible neural network. It has 1 layer, that layer has 1 neuron, and the input is just 1 value.

#### Age Job Type Model

In [56]:
input1 = tf.keras.layers.Input(shape=(1,))
input2 = tf.keras.layers.Input(shape=(1,))
input3 = tf.keras.layers.Input(shape=(1,))
input4 = tf.keras.layers.Input(shape=(1,))
input5 = tf.keras.layers.Input(shape=(1,))


merged = tf.keras.layers.Concatenate(axis=1)([input1, input2, input3, input4, input5])
dense1 = tf.keras.layers.Dense(5, input_dim=1, activation=tf.keras.activations.sigmoid, use_bias=True)(merged)
output = tf.keras.layers.Dense(1, activation=tf.keras.activations.relu, use_bias=True)(dense1)
age_job_category_model = tf.keras.models.Model([input1, input2, input3, input4, input5], output)
age_job_category_model.compile(optimizer='sgd', loss='mean_squared_error')
age_job_category_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_31 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_32 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_33 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_34 (InputLayer)          [(None, 1)]          0           []                               
                                                                                            

In [57]:
# Train data
input1 = train.loc[:,['age']]
input2 = train.loc[:,['Job_Clerical or administrative support']]
input3 = train.loc[:,['Job_Manufacturing, construction, maintenance, or farming']]
input4 = train.loc[:,['Job_Professional, managerial, or technical']]
input5 = train.loc[:,['Job_Sales or service']]

input1 = np.asarray(input1).astype('float32')
input2 = np.asarray(input2).astype('float32')
input3 = np.asarray(input3).astype('float32')
input4 = np.asarray(input4).astype('float32')
input5 = np.asarray(input5).astype('float32')

y = train.loc[:,['wfh_expect']]
y = np.asarray(y).astype('float32')

# Validation data
v_input1 = validation.loc[:,['age']]
v_input2 = validation.loc[:,['Job_Clerical or administrative support']]
v_input3 = validation.loc[:,['Job_Manufacturing, construction, maintenance, or farming']]
v_input4 = validation.loc[:,['Job_Professional, managerial, or technical']]
v_input5 = validation.loc[:,['Job_Sales or service']]

v_input1 = np.asarray(v_input1).astype('float32')
v_input2 = np.asarray(v_input2).astype('float32')
v_input3 = np.asarray(v_input3).astype('float32')
v_input4 = np.asarray(v_input4).astype('float32')
v_input5 = np.asarray(v_input5).astype('float32')

v_y = validation.loc[:,['wfh_expect']]
v_y = np.asarray(v_y).astype('float32')

age_job_category_model.fit([input1, input2, input3, input4, input5],y, 
                        batch_size=36, epochs=100, validation_data=([v_input1, v_input2, v_input3, v_input4, v_input5], v_y))


ValueError: could not convert string to float: 'Yes'

In [ ]:
# Test data
t_input1 = test.loc[:,['age']]
t_input2 = test.loc[:,['Job_Clerical or administrative support']]
t_input3 = test.loc[:,['Job_Manufacturing, construction, maintenance, or farming']]
t_input4 = test.loc[:,['Job_Professional, managerial, or technical']]
t_input5 = test.loc[:,['Job_Sales or service']]

t_input1 = np.asarray(t_input1).astype('float32')
t_input2 = np.asarray(t_input2).astype('float32')
t_input3 = np.asarray(t_input3).astype('float32')
t_input4 = np.asarray(t_input4).astype('float32')
t_input5 = np.asarray(t_input5).astype('float32')

In [ ]:
# prediction
age_job_category_model.predict([t_input1, t_input2, t_input3, t_input4, t_input5])

10/10 [==============================] - 0s 1ms/step


array([[0.49092886],
       [0.48997596],
       [0.49107453],
       [0.48268172],
       [0.48730424],
       [0.48997596],
       [0.4777086 ],
       [0.49102786],
       [0.49134395],
       [0.49127498],
       [0.49121407],
       [0.49127498],
       [0.49105105],
       [0.49133167],
       [0.49066594],
       [0.49107453],
       [0.48609582],
       [0.4900615 ],
       [0.49136767],
       [0.49134395],
       [0.49121407],
       [0.48879763],
       [0.49107453],
       [0.4858593 ],
       [0.49128994],
       [0.4913027 ],
       [0.48730424],
       [0.48663548],
       [0.4894394 ],
       [0.4911867 ],
       [0.48853442],
       [0.48997596],
       [0.4912577 ],
       [0.49131367],
       [0.4897448 ],
       [0.49107453],
       [0.49136326],
       [0.4910241 ],
       [0.48465696],
       [0.49061707],
       [0.4908177 ],
       [0.49115488],
       [0.48947564],
       [0.4911867 ],
       [0.49132302],
       [0.4858578 ],
       [0.49034497],
       [0.491

#### Degree or Pro Environment Model

In [ ]:
input1 = tf.keras.layers.Input(shape=(1,))
input2 = tf.keras.layers.Input(shape=(1,))
input3 = tf.keras.layers.Input(shape=(1,))
input4 = tf.keras.layers.Input(shape=(1,))
input5 = tf.keras.layers.Input(shape=(1,))

merged = tf.keras.layers.Concatenate(axis=1)([input1, input2, input3, input4, input5])
dense1 = tf.keras.layers.Dense(5, input_dim=1, activation=tf.keras.activations.sigmoid, use_bias=True)(merged)
output = tf.keras.layers.Dense(1, activation=tf.keras.activations.relu, use_bias=True)(dense1)
degree_environment_model = tf.keras.models.Model([input1, input2, input3, input4, input5], output)
degree_environment_model.compile(optimizer='sgd', loss='mean_squared_error')
degree_environment_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_22 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_23 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_24 (InputLayer)          [(None, 1)]          0           []                               
                                                                                            

In [ ]:
# Training Data
input1 = train.loc[:,['age']]
input2 = train.loc[:,['Workload_increased']]
input3 = train.loc[:,['Workload_decreased']]
input4 = train.loc[:,['Gradutae_degree']]
input5 = train.loc[:,['ProEnvironment']]

input1 = np.asarray(input1).astype('float32')
input2 = np.asarray(input2).astype('float32')
input3 = np.asarray(input3).astype('float32')
input4 = np.asarray(input4).astype('float32')
input5 = np.asarray(input5).astype('float32')

y = train.loc[:,['wfh_expect']]
y = np.asarray(y).astype('float32')

# Validation Data
v_input1 = validation.loc[:,['age']]
v_input2 = validation.loc[:,['Workload_increased']]
v_input3 = validation.loc[:,['Workload_decreased']]
v_input4 = validation.loc[:,['Gradutae_degree']]
v_input5 = validation.loc[:,['ProEnvironment']]

v_input1 = np.asarray(v_input1).astype('float32')
v_input2 = np.asarray(v_input2).astype('float32')
v_input3 = np.asarray(v_input3).astype('float32')
v_input4 = np.asarray(v_input4).astype('float32')
v_input5 = np.asarray(v_input5).astype('float32')

v_y = validation.loc[:,['wfh_expect']]
v_y = np.asarray(v_y).astype('float32')

degree_environment_model.fit([input1, input2, input3, input4, input5],y, batch_size=36, epochs=100, 
                        validation_data=([v_input1, v_input2, v_input3, v_input4, v_input5], v_y))

Epoch 1/100
23/23 [==============================] - 0s 4ms/step - loss: 0.2506 - val_loss: 0.2480
Epoch 2/100
23/23 [==============================] - 0s 2ms/step - loss: 0.2506 - val_loss: 0.2467
Epoch 3/100
23/23 [==============================] - 0s 2ms/step - loss: 0.2505 - val_loss: 0.2495
Epoch 4/100
23/23 [==============================] - 0s 2ms/step - loss: 0.2504 - val_loss: 0.2484
Epoch 5/100
23/23 [==============================] - 0s 2ms/step - loss: 0.2503 - val_loss: 0.2495
Epoch 6/100
23/23 [==============================] - 0s 2ms/step - loss: 0.2502 - val_loss: 0.2474
Epoch 7/100
23/23 [==============================] - 0s 2ms/step - loss: 0.2506 - val_loss: 0.2504
Epoch 8/100
23/23 [==============================] - 0s 2ms/step - loss: 0.2504 - val_loss: 0.2473
Epoch 9/100
23/23 [==============================] - 0s 1ms/step - loss: 0.2502 - val_loss: 0.2487
Epoch 10/100
23/23 [==============================] - 0s 2ms/step - loss: 0.2503 - val_loss: 0.2477
Epoch 11/

In [ ]:
# Test data
t_input1 = test.loc[:,['age']]
t_input2 = test.loc[:,['Workload_increased']]
t_input3 = test.loc[:,['Workload_decreased']]
t_input4 = test.loc[:,['Gradutae_degree']]
t_input5 = test.loc[:,['ProEnvironment']]

t_input1 = np.asarray(t_input1).astype('float32')
t_input2 = np.asarray(t_input2).astype('float32')
t_input3 = np.asarray(t_input3).astype('float32')
t_input4 = np.asarray(t_input4).astype('float32')
t_input5 = np.asarray(t_input5).astype('float32')

In [ ]:
# prediction
degree_environment_model.predict([t_input1, t_input2, t_input3, t_input4, t_input5])

10/10 [==============================] - 0s 628us/step


array([[0.48788524],
       [0.487656  ],
       [0.4879204 ],
       [0.48520166],
       [0.48683208],
       [0.487656  ],
       [0.48335648],
       [0.48789835],
       [0.48796362],
       [0.48795462],
       [0.4879452 ],
       [0.48795462],
       [0.48795402],
       [0.4879613 ],
       [0.48782486],
       [0.4879204 ],
       [0.48660904],
       [0.48767382],
       [0.48796582],
       [0.48796362],
       [0.4879452 ],
       [0.48731196],
       [0.4879204 ],
       [0.48579633],
       [0.48795676],
       [0.48795855],
       [0.48683208],
       [0.48660684],
       [0.48740232],
       [0.48794067],
       [0.48726445],
       [0.487656  ],
       [0.4879521 ],
       [0.48795992],
       [0.48759186],
       [0.4879204 ],
       [0.48796546],
       [0.4879105 ],
       [0.4857415 ],
       [0.48782074],
       [0.487867  ],
       [0.48793507],
       [0.48751497],
       [0.48794067],
       [0.48796117],
       [0.48633897],
       [0.4877534 ],
       [0.487